# Executive Summary



## Part 1: Linux Commands - Setup

### The following commands are executed once:

#### set up project directory and branches
ls

cd w205

git clone https://github.com/mids-w205-martin-mims/project-3-edakavlakoglu.git

cd project-3-edakavlakoglu

git branch assignment

git checkout assignment

#### copy yml file in project directory
cp ~/w205/course-content/12-Querying-Data-II/docker-compose.yml .

#### check yml file 
cat docker-compose.yml

#### copy game_api python file and edit to include 'buy_a_sword' and 'join_guild' events
cp ~/w205/course-content/12-Querying-Data-II/game_api.py .

### The following commands are executed every time I start a cluster:

#### check for old containers
docker ps -a

#### start up cluster
docker-compose up -d 

docker-compose ps

#### create topic events
docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --bootstrap-server localhost:29092

#### run flask
docker-compose exec mids env FLASK_APP=/w205/project-3-edakavlakoglu/game_api.py flask run --host 0.0.0.0

#### execute a bash shell in spark container in new terminal window
docker-compose exec spark bash

#### create symbolic link in spark container
ln -s /w205 w205

#### exit container
exit

#### run a jupyter notebook with pyspark kernel
docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark

#### check jupyter notebook exists
ls

#### start up jupyter notebook in incognito window (IP = 34.127.126.57) 
http://34.127.126.57:8888/?token=daa6b577712346d0f1e9ceb759ee67508d35af6d2aa472a3

#### Run the Apache Bench commands in a separate terminal window:

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_sword

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/purchase_a_sword

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/buy_a_sword

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/join_guild

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/buy_a_sword

docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/join_guild

## Part 2: Exploring JSON file via Spark SQL

In [8]:
import json
from pyspark.sql import Row
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType

In [9]:
spark

In [10]:
sc

<SparkContext master=local[*] appName=PySparkShell>

## Batch Mode

In [11]:
# Read json data from kafka

@udf('boolean')
def is_purchase(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_sword':
        return True
    return False

In [12]:
# Create a spark dataframe containing the extracted json data
raw_events = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "events") \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load()

In [13]:
raw_events.show()

+----+--------------------+------+---------+------+--------------------+-------------+
| key|               value| topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------+---------+------+--------------------+-------------+
|null|[7B 22 48 6F 73 7...|events|        0|     0|2021-11-19 03:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     1|2021-11-19 03:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     2|2021-11-19 03:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     3|2021-11-19 03:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     4|2021-11-19 03:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     5|2021-11-19 03:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     6|2021-11-19 03:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     7|2021-11-19 03:04:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0

In [14]:
purchase_events = raw_events \
    .select(raw_events.value.cast('string').alias('raw'),
    raw_events.timestamp.cast('string')) \
    .filter(is_purchase('raw'))

In [15]:
purchase_events.show()

+--------------------+--------------------+
|                 raw|           timestamp|
+--------------------+--------------------+
|{"Host": "user1.c...|2021-11-19 03:04:...|
|{"Host": "user1.c...|2021-11-19 03:04:...|
|{"Host": "user1.c...|2021-11-19 03:04:...|
|{"Host": "user1.c...|2021-11-19 03:04:...|
|{"Host": "user1.c...|2021-11-19 03:04:...|
|{"Host": "user1.c...|2021-11-19 03:04:...|
|{"Host": "user1.c...|2021-11-19 03:04:...|
|{"Host": "user1.c...|2021-11-19 03:04:...|
|{"Host": "user1.c...|2021-11-19 03:04:...|
|{"Host": "user1.c...|2021-11-19 03:04:...|
|{"Host": "user2.a...|2021-11-19 03:05:...|
|{"Host": "user2.a...|2021-11-19 03:05:...|
|{"Host": "user2.a...|2021-11-19 03:05:...|
|{"Host": "user2.a...|2021-11-19 03:05:...|
|{"Host": "user2.a...|2021-11-19 03:05:...|
|{"Host": "user2.a...|2021-11-19 03:05:...|
|{"Host": "user2.a...|2021-11-19 03:05:...|
|{"Host": "user2.a...|2021-11-19 03:05:...|
|{"Host": "user2.a...|2021-11-19 03:05:...|
|{"Host": "user2.a...|2021-11-19

In [16]:
# Filter out only the purchase sword extracted events
extracted_purchase_events = purchase_events \
    .rdd \
    .map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.raw))) \
    .toDF()

In [17]:
extracted_purchase_events.printSchema()

root
 |-- Accept: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- User-Agent: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- sword_size: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [18]:
extracted_purchase_events.show()

+------+-----------------+---------------+--------------+----------+--------------------+
|Accept|             Host|     User-Agent|    event_type|sword_size|           timestamp|
+------+-----------------+---------------+--------------+----------+--------------------+
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|us

In [19]:
# Write the purchase sword extracted events to parquet files stored in hdfs
extracted_purchase_events \
    .write \
    .mode('overwrite') \
    .parquet('/tmp/purchases')

In [20]:
# Read the parquet files from hdfs into another spark dataframe
purchases = spark.read.parquet('/tmp/purchases')

In [21]:
purchases.show()

+------+-----------------+---------------+--------------+----------+--------------------+
|Accept|             Host|     User-Agent|    event_type|sword_size|           timestamp|
+------+-----------------+---------------+--------------+----------+--------------------+
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|us

In [22]:
# Register as a temp table to allow in memory sql against the dataframe
purchases.registerTempTable('purchases')

In [23]:
purchases_by_example2 = spark.sql("select * from purchases where host='user1.comcast.com'")

In [24]:
purchases_by_example2.show()

+------+-----------------+---------------+--------------+----------+--------------------+
|Accept|             Host|     User-Agent|    event_type|sword_size|           timestamp|
+------+-----------------+---------------+--------------+----------+--------------------+
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|     small|2021-11-19 03:04:...|
|   */*|us

In [25]:
# Convert to pandas
newdf = purchases_by_example2.toPandas()

In [26]:
newdf.describe()

,Accept,Host,User-Agent,event_type,sword_size,timestamp
count,1620,1620,1620,1620,1620,1620
unique,1,1,1,1,1,1620
top,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,small,2021-11-19 03:58:43.066
freq,1620,1620,1620,1620,1620,1


In [27]:
newdf

,Accept,Host,User-Agent,event_type,sword_size,timestamp
0,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,small,2021-11-19 03:04:54.19
1,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,small,2021-11-19 03:04:54.197
2,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,small,2021-11-19 03:04:54.206
3,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,small,2021-11-19 03:04:54.212
4,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,small,2021-11-19 03:04:54.219
5,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,small,2021-11-19 03:04:54.223
6,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,small,2021-11-19 03:04:54.235
7,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,small,2021-11-19 03:04:54.255
8,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,small,2021-11-19 03:04:54.267
9,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,small,2021-11-19 03:04:54.272


## Streaming Mode

Read live streaming data and write it to hadoop hdfs in parquet format

In [28]:
def purchase_sword_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
         StructField("sword_size", StringType(), True),
    ])

In [29]:
@udf('boolean')
def is_sword_purchase(event_as_json):
    """udf for filtering events
    """
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_sword':
        return True
    return False

In [30]:
raw_events = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "events") \
    .load()

In [31]:
sword_purchases = raw_events \
    .filter(is_sword_purchase(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                    purchase_sword_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')

NameError: name 'from_json' is not defined

In [ ]:
sink = sword_purchases \
    .writeStream \
    .format("parquet") \
    .option("checkpointLocation", "/tmp/checkpoints_for_sword_purchases") \
    .option("path", "/tmp/sword_purchases") \
    .trigger(processingTime="10 seconds") \
    .start()

docker-compose exec cloudera hive

create external table if not exists default.sword_purchases (Accept string, Host string, `User_Agent` string, event_type string, sword_size string, timestamp string) stored as parquet location '/tmp/sword_purchases'  tblproperties ("parquet.compress"="SNAPPY");

docker-compose exec presto presto --server presto:8080 --catalog hive --schema default

In [ ]:
sink.stop()

## Part W: Simple Analytics - ask 1-2 business questions

## Part X: Linux Commands - Tear Down

#### check hadoop hdfs writes
docker-compose exec cloudera hadoop fs -ls /tmp/

docker-compose exec cloudera hadoop fs -ls /tmp/purchases/

#### tear down cluster
docker-compose down

docker-compose ps